In [1]:
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import numpy as np
import uproot
import os
import glob
import awkward as ak
from scipy.stats import norm

In [2]:
setup_path ="/home/haue/repositories/Masters/setup/" 
data_path = "/home/haue/repositories/Masters/data/singleprotons/Al/inclbetacut/"
twop_path = "/home/haue/repositories/Masters/data/twoproton/Al/"
Al_files = glob.glob(os.path.join(data_path, "*mlio.root"))
twop_files = glob.glob(os.path.join(twop_path, "*mlio.root"))


frac = 0.9542011788469517 # (Na21/(p+Na21)) amount of energy the daughter nucleus gets of the total energy released in the decay
frac_err = 1.032071836027589e-10 # uncertainty of the fraction
psep = 5.50410 # proton separation energy of 22Mg
psep_err = 0.00016 # error of proton separation energy of 22Mg
scale = 3.326542500626603 # difference in solid angle coverage of two proton events up to 4pi
sac4 = 6.795917343263473 # solid angle coverage of id3
sac3 = 6.572831132771112
sac2 = 6.068117863153485

def weighted_mean(values, errors, inflate=True, returnchi=False):
    values = np.asarray(values)
    errors = np.asarray(errors)
    weights = 1.0 / (errors**2)

    mean = np.sum(weights * values) / np.sum(weights)
    err_mean = np.sqrt(1.0 / np.sum(weights))

    # chi2
    chi2 = np.sum(((values - mean)/errors)**2)
    dof = len(values) - 1
    chi2_red = chi2 / dof if dof > 0 else np.nan

    # inflate if chi2_red > 1
    if inflate and chi2_red > 1:
        err_mean *= np.sqrt(chi2_red)
    if returnchi:
        return mean, err_mean, chi2, dof, chi2_red
    else:
        return mean, err_mean

In [3]:
E2_id3 = np.array([])
for batch in uproot.iterate(Al_files, expressions=["E"],
                            cut="(id==3)"):
    E2_id3=np.append(E2_id3, ak.flatten(batch.E))
E2p_id3 = np.array([])
for batch in uproot.iterate(twop_files, expressions=["E"],
                            cut="(id==3)"):
    E2p_id3=np.append(E2p_id3, ak.flatten(batch.E))
    
E2_id2 = np.array([])
for batch in uproot.iterate(Al_files, expressions=["E"],
                            cut="(id==2)"):
    E2_id2=np.append(E2_id2, ak.flatten(batch.E))
E2p_id2 = np.array([])
for batch in uproot.iterate(twop_files, expressions=["E"],
                            cut="(id==2)"):
    E2p_id2=np.append(E2p_id2, ak.flatten(batch.E))
E2p_tot = np.array([])
for batch in uproot.iterate(twop_files, expressions=["E"]):
    E2p_tot=np.append(E2p_tot, ak.flatten(batch.E))
    



In [4]:
a1 = weighted_mean([268*100/sac2, 270*100/sac3], [18*100/sac2, 17*100/sac3])
a0 = weighted_mean([43*100/sac2, 59*100/sac3], [9*100/sac2, 10*100/sac3])
tot_a_counts = a1[0]+a0[0]
tot_a_counts_err = np.sqrt(a1[1]**2+a0[1]**2)
a1branch = a1[0]/(a1[0]+a0[0])
a1branch_err = np.sqrt( ((a0[0]*a1[1])**2 + (a1[0]*a0[1])**2) ) / (a1[0]+a0[0])**2
a0branch = a0[0]/(a1[0]+a0[0])
a0branch_err = np.sqrt( ((a1[0]*a0[1])**2 + (a0[0]*a1[1])**2) ) / (a1[0]+a0[0])**2
a1branch, a1branch_err, a0branch, a0branch_err, a1[0], a1[1]

(0.8411855322534131,
 0.018752598373867103,
 0.158814467746587,
 0.018752598373867103,
 4241.149039308798,
 194.94378083932747)

In [5]:
y1p_id3, _ = np.histogram(E2_id3, bins=np.arange(0,9000+10,10))
#y2p_id3, _ = np.histogram(E2p_id3, bins=np.arange(0,9000+10,10))
y1p_id2, _ = np.histogram(E2_id2, bins=np.arange(0,1000+10,10)) #to find the missing counts from id3 spectrum
#y2p_id2, _ = np.histogram(E2p_id2, bins=np.arange(0,1000+10,10)) # to find the missing counts from id3 spectrum
y2p, _ = np.histogram(E2p_tot, bins=np.arange(0,9000+10,10)) #to find the missing counts from id3 spectrum

#U2 counts 268(18) 43(9)
#U3 counts 270(17) 59(10)

counts_1p_id3 = np.sum(y1p_id3); counts_1p_id3_err = np.sqrt(np.sum(y1p_id3))
#counts_2p_id3 = np.sum(y2p_id3); counts_2p_id3_err = np.sqrt(np.sum(y2p_id3))
counts_1p_id2 = np.sum(y1p_id2); counts_1p_id2_err = np.sqrt(np.sum(y1p_id2))
#counts_2p_id2 = np.sum(y2p_id2); counts_2p_id2_err = np.sqrt(np.sum(y2p_id2))
counts_2p = np.sum(y2p); counts_2p_err = np.sqrt(np.sum(y2p))


relative_diff = 11122/9083 # relative peaksizes of 1.245 peak of id3/id2 186, 290
#relative_diff_err = 0.05
relative_diff_err = np.sqrt((186 / 9083)**2 + (11122 * 290 / 9083**2)**2+0.05**2)
#print(relative_diff_err)
rel_SA_3 = 100/sac4
rel_SA_2 = 100/sac4

relative_solidangle_3 = (100/scale)
relative_solidangle_2 = (100/scale)
#tot_1p_counts = (counts_1p_id3* rel_SA_3-counts_2p_id3*scale + counts_1p_id2*relative_diff* rel_SA_2-counts_2p_id2*scale*relative_diff ) # total 1p counts in id3 where the region below 1MeV has been estimated from id2
#tot_1p_counts_err = np.sqrt((rel_SA_3*counts_1p_id3_err)**2 + (scale*counts_2p_id3_err)**2 + (rel_SA_2*relative_diff*counts_1p_id2_err)**2 + (scale*relative_diff*counts_2p_id2_err)**2 + (rel_SA_2*counts_1p_id2*relative_diff_err)**2 + (scale*counts_2p_id2*relative_diff_err)**2)

#tot_2p_counts = ((counts_2p_id3*scale+counts_2p_id2*scale*relative_diff)) /2 # divided by 2 because 1 2p event is two protons
#tot_2p_counts_err = 0.5*np.sqrt((scale*counts_2p_id3_err)**2 + (scale*relative_diff*counts_2p_id2_err)**2 + (scale*counts_2p_id2*relative_diff_err)**2)

tot_p_counts = (counts_1p_id3* rel_SA_3+ counts_1p_id2*relative_diff* rel_SA_2)
tot_p_counts_err = np.sqrt((rel_SA_3*counts_1p_id3_err)**2 + (rel_SA_2*relative_diff*counts_1p_id2_err)**2 +(rel_SA_2*counts_1p_id2*relative_diff_err)**2)

tot_2p_counts = (counts_2p*scale)/2
tot_2p_counts_err = (counts_2p_err*scale)/2

tot_p1_counts = tot_p_counts-tot_2p_counts-tot_a_counts
tot_p1_counts_err = np.sqrt(tot_p_counts_err**2+tot_2p_counts_err**2+tot_a_counts_err**2)


tot_g_counts = 795181# weighted mean calculated from counts in 1247 peak in G1 & G2 with beta threshold
tot_g_counts_err = 125382
tot = tot_p1_counts+tot_2p_counts+tot_g_counts+tot_a_counts
tot_err = np.sqrt(tot_p1_counts_err**2+tot_2p_counts_err**2+tot_g_counts_err**2+tot_a_counts_err**2)


P1, P2, G, A = tot_p1_counts, tot_2p_counts, tot_g_counts, tot_a_counts
sP1, sP2, sG, sA = tot_p1_counts_err, tot_2p_counts_err, tot_g_counts_err, tot_a_counts_err
T = P1 + P2 + G + A

print(f"Branching ratio 1p: {P1/T*100:.2f} +/- {100*np.sqrt((T-P1)**2*sP1**2 + P1**2*(sP2**2 + sG**2 + sA**2))/T**2:.2f}")
print(f"Branching ratio 2p: {P2/T*100:.2f} +/- {100*np.sqrt((T-P2)**2*sP2**2 + P2**2*(sP1**2 + sG**2 + sA**2))/T**2:.2f}")
print(f"Branching ratio g: {G/T*100:.2f} +/- {100*np.sqrt((T-G)**2*sG**2 + G**2*(sP1**2 + sP2**2 + sA**2))/T**2:.2f}")
print(f"Branching ratio alpha: {A/T*100:.2f} +/- {100*np.sqrt((T-A)**2*sA**2 + A**2*(sP1**2 + sP2**2 + sG**2))/T**2:.2f}")
print(tot_p1_counts, tot_p1_counts_err)
print(tot_2p_counts, tot_2p_counts_err)
print(tot_a_counts, tot_a_counts_err)
print(tot_g_counts, tot_g_counts_err)

Branching ratio 1p: 44.14 +/- 3.83
Branching ratio 2p: 0.75 +/- 0.07
Branching ratio g: 54.76 +/- 3.92
Branching ratio alpha: 0.35 +/- 0.03
641009.589188178 9732.302385908817
10914.385944555885 134.73523799067337
5041.871117240188 221.99535717119713
795181 125382


In [9]:
BR_alpha = A/T
BR_alpha_err = np.sqrt((T-A)**2*sA**2 + A**2*(sP1**2 + sP2**2 + sG**2))/T**2
BR_a1 = BR_alpha * a1branch
BR_a1_err = BR_a1 * np.sqrt((BR_alpha_err/BR_alpha)**2 + (a1branch_err/a1branch)**2)

BR_a0 = BR_alpha * a0branch
BR_a0_err = BR_a0 * np.sqrt((BR_alpha_err/BR_alpha)**2 + (a0branch_err/a0branch)**2)
print(f"Absolute branching ratio a1: {BR_a1*100:.2f} +/- {BR_a1_err*100:.2f}")
print(f"Absolute branching ratio a0: {BR_a0*100:.2f} +/- {BR_a0_err*100:.2f}")

#261 (16) 2p 0+ counts
#856 (29) 2p 2+ counts
zeroplus = 261*scale
zeroplus_err = 16*scale
twoplus = 856*scale
twoplus_err = 29*scale
BR_2p = P2 / T
BR_2p_err = np.sqrt(((T - P2)**2 * sP2**2 + P2**2 * (sA**2 + sP1**2 + sG**2))) / T**2

BR_zeroplus = zeroplus / T
BR_zeroplus_err = np.sqrt(((T - zeroplus)**2 * zeroplus_err**2 + zeroplus**2 * (sA**2 + sP1**2 + sP2**2 + sG**2))) / T**2

BR_twoplus = twoplus / T
BR_twoplus_err = np.sqrt(((T - twoplus)**2 * twoplus_err**2 + twoplus**2 * (sA**2 + sP1**2 + sP2**2 + sG**2))) / T**2

print(f"Absolute branching ratio 0+: {BR_zeroplus*100:.2f} +/- {BR_zeroplus_err*100:.2f}")
print(f"Absolute branching ratio 2+: {BR_twoplus*100:.2f} +/- {BR_twoplus_err*100:.2f}")

Absolute branching ratio a1: 0.29 +/- 0.03
Absolute branching ratio a0: 0.06 +/- 0.01
Absolute branching ratio 0+: 0.06 +/- 0.01
Absolute branching ratio 2+: 0.20 +/- 0.02


In [10]:
totIASbr = 0.35+0.57+0.26
totIASbr_err = np.sqrt(3**2+5**2+2**2)
totIASbr, totIASbr_err

(1.18, 6.164414002968976)

In [8]:
def rel(totc,totc_e,c, c_e):
    br = (c/totc) *100
    br_e = br*np.sqrt((c_e/c)**2+(totc_e/totc)**2)
    return br,br_e

def weighted_mean(values, errors):
    values = np.asarray(values)
    errors = np.asarray(errors)
    weights = 1.0 / (errors**2)
    mean = np.sum(weights * values) / np.sum(weights)
    err_mean = np.sqrt(1.0 / np.sum(weights))
    return mean, err_mean

#print(rel(tot,tot_err,11122*relative_solidangle,186*relative_solidangle))

In [9]:

ps3 = [1249,1167,1311,2486,2396,2563,5456,3880,1670,     6517,7843, 1850, 8158]
cs3 = [11129,306,596,3741,66,130,144,363,496,              119,83, 163, 26]
ce3 = [630,33,47,371,8,23,11,39,321,                    11,9, 56, 5]

cs2 = [9084,118,179,210,352,   1402,256,3242,1919,358,        234,78, 205]
ce2 = [973,21,10,32,43,          55,27,103,288,45,         15,10, 48]
ps2 = [1249,1167,1311,5456,3880,  996,775,693,452,1670,    6522,7844,1850]

cs1 = [8604,161,116,225,335,1186,260,2237,332,      222, 95,148,17]
ce1 = [922,17,6,29,40,66,30,55, 42,                  15,10,46,4]
ps1 = [1249,1167,1311,5456,3880,996,775,693,1670,      6524, 7847,1850,8158]




"""
for c,p,e in zip(cs1,ps1,ce1):
    print(p, rel(cs1[0], ce1[0], c,e))

    
print("="*60)
for c,p,e in zip(cs2,ps2,ce2):
    print(p, rel(cs2[0], ce2[0], c,e))


print("="*60)
for c,p,e in zip(cs3,ps3,ce3):
    print(p, rel(cs3[0], ce3[0], c,e))
print("="*60)
print("="*60)

"""
#print(ps1[-1], rel(cs1[0], ce1[0], cs1[-1],ce1[-1]))
#print(ps2[-1], rel(cs2[0], ce2[0], cs2[-1],ce2[-1]))
#print(ps3[-1], rel(cs3[0], ce3[0], cs3[-1],ce3[-1]))

absscale, absscale_err = rel(tot,tot_err,cs3[0]*rel_SA_3,ce3[0]*rel_SA_3)
absscale /= 100
absscale_err /= 100
p1249 = weighted_mean([100,100,100],[15.154636268107783,15.147840116567826,8.005701718888039])
p1167 = weighted_mean([1.8712226871222686,1.2989872302950243,2.749573187168658],[0.28150809457245957,0.26981683472575363,0.3348919783281118])
p1311 = weighted_mean([1.3482101348210134,1.9704975781594012,5.355377841674904], [0.16042305013423053,0.2380460105115358,0.5198666502826514])

p5456 = weighted_mean([2.615062761506276,2.3117569352708056,1.2939167939617218], [0.3590475232387749,0.4305879712403336,0.12302301591640319])
p3880 = weighted_mean([3.893537889353789,3.8749449581682076,3.2617485847785064], [0.4894032231917866,0.6295527147416979,0.3961042877995525])
p996 = weighted_mean([13.784286378428638, 15.433729634522237], [1.6644197921713146,1.760515748159705]) # id1,id2
p775 = weighted_mean([3.0218503021850305, 2.8181417877586967], [0.475850369740362, 0.42362689649911534]) # id1,id2
p693 = weighted_mean([25.99953509995351, 35.68912373403787], [2.858488495145724, 3.98732627245805]) # id1,id2
p452 = weighted_mean(21.125055041831793, 3.895056234281918) #id2
p2486 = weighted_mean(33.614880043130555, 3.838507067582582) #id3
p2396 =weighted_mean(0.5930451972324557, 0.07933726050380133) #id3
p2563 =weighted_mean(1.1681193278821098, 0.21698846463602584) #id3
p1670 = weighted_mean([3.8586703858670384,3.9409951563188024,4.456824512534819], [0.6397359461924,0.65083624940473,2.8953693647207346])
p6417 =weighted_mean([2.580195258019526,2.5759577278731833], [0.3268663027234486,0.3215512607212417]) #id1&id2 ---- id3,0.8985533291400845,0.1867706653714041
p7844 =weighted_mean([1.104137610413761,0.8586525759577279,0.7457992631862701], [0.1658541214558144,0.1434474355688095,0.09122694469567033]) 
p1850 =weighted_mean([1.7201301720130173,2.2567151034786437,1.4646419264983377], [0.5655188181378092,0.5810651918919552,0.509974892622362]) #--> beviset ligger i 332 koincidens spektret
p8158 =weighted_mean([0.19758251975825197,0.23362386557642195], [0.03793562587642831,0.04683375509354919]) #--> id3 & id1


peaks = [p1249,p1167,p1311,p5456,p3880,p996,p775,p693,p452,p2486,p2396,p2563,p1670,p6417,p7844,p1850,p8158]
peakvals = [1249,1167,1311,5456,3880,996,775,693,452,2486,2396,2563,1673,6517,7844,1850,8158]
brs = []
for (mean, err),p in zip(peaks,peakvals):
    abs_mean = mean * absscale
    abs_err = abs_mean * np.sqrt((err / mean)**2 + (absscale_err / absscale)**2)
    brs.append(abs_mean)
    print(f"E={p} - {abs_mean:.2f} +/- {abs_err:.2f}")
print(f"Total branching ratio accounted for: {np.sum(brs):.2f} %")
print(f"Total 1-p branching ratio {tot_p1_counts/tot*100:.2f} % +/- {100 * np.sqrt((tot_p1_counts_err / tot)**2 + (tot_p1_counts * tot_err / tot**2)**2):.2f} %")
print(f"Missing branching ratio: {tot_p1_counts/tot*100-np.sum(brs):.2f} %")
#"""


E=1249 - 11.28 +/- 1.37
E=1167 - 0.21 +/- 0.03
E=1311 - 0.20 +/- 0.03
E=5456 - 0.17 +/- 0.02
E=3880 - 0.40 +/- 0.05
E=996 - 1.64 +/- 0.22
E=775 - 0.33 +/- 0.05
E=693 - 3.30 +/- 0.43
E=452 - 2.38 +/- 0.50
E=2486 - 3.79 +/- 0.58
E=2396 - 0.07 +/- 0.01
E=2563 - 0.13 +/- 0.03
E=1673 - 0.44 +/- 0.07
E=6517 - 0.29 +/- 0.04
E=7844 - 0.09 +/- 0.01
E=1850 - 0.20 +/- 0.04
E=8158 - 0.02 +/- 0.00
Total branching ratio accounted for: 24.96 %
Total 1-p branching ratio 44.14 % +/- 3.88 %
Missing branching ratio: 19.19 %


In [ ]:
"""
logft's calculated from nndc's tool using T½ = 91.1(5)ms Q-value 17.6(4) and then using the energy levels from the decay scheme and the summed transition intensities based upon this script
6229 - 5.38(9)
6310 - 5.46(11)
6879 - 5.50(9)
7146 - 4.66(10)
7584 - 5.96(11)
8348 - 6.63(12)
8442 - 4.80(12)
9902 - 5.27(13)
7775 - 6.28(13)
"""

0.05477225575051661